<a href="https://colab.research.google.com/github/DavoodSZ1993/RL/blob/main/05_MC_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import gym

env = gym.make('Blackjack-v0')

In [2]:
state_space_size = (33, 12, 2)
policy = np.zeros(state_space_size, dtype=int)

In [3]:
def observation_clean(observation):
  return (observation[0], observation[1], int(observation[2]))

In [4]:
def run_episode_exploring_start(policy, env=env):
  steps = []
  observation = observation_clean(env.reset())
  done = False
  steps.append(((None, None) + (observation, 0))) # State, Action, Next State, Reward
  start = True

  while not done:
    if start:
      action = np.random.binomial(n=1, p=0.5)
      start = False
    else:
      action = policy[observation]

    observation_action = (observation, action)
    observation, reward, done, info = env.step(action)
    observation = observation_clean(observation)
    steps.append(observation_action + (observation, int(reward)))
  return steps

In [10]:
observation = observation_clean(env.reset())
action = np.random.binomial(n=1, p=0.5)
action

0